In [122]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import IPython
import re
import collections

sys.path.append("../src/utils")
sys.path.append("../src")

In [76]:
%load_ext autoreload
%autoreload 1
%aimport datasets
%aimport general
%aimport training
%aimport generation
%aimport evaluation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
from general import *

In [78]:
openai_token = open('../config/openai-token.txt').read().strip()
os.environ["OPENAI_API_KEY"] = openai_token

In [79]:
actions_path = '../data/actions.pkl'

import pickle
with open(actions_path, 'rb') as f:
    actions = pickle.load(f)

In [80]:
from transformers import GPT2Tokenizer, AutoTokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_tokens([tok for tok in datasets.CONTROL_TOKENS.values() if tok != 'eos'])
temp = lfilter(lambda a: a.user == 'p', actions[0])
model = generation.SamplingModel(sleep = 0, tokenizer=tokenizer, untokenized=[''.join([datasets.action_to_string(random.choice(temp)) for _ in range(random.randint(1,3))]) for _ in range(10)])

In [81]:
pipeline = generation.Pipeline(model=model, tokenizer=tokenizer, agents=['p'])

In [160]:
ctxs, refs = evaluation.sample_qa_like(actions, agents=['p'], context_size=lambda i: 20, n=1000)

In [161]:
eval_ctxs_path = '../data/eval_ctxs.pkl'

import pickle
with open(eval_ctxs_path, 'wb') as f:
    pickle.dump(ctxs, f)

In [155]:
for ctx, ref in list(zip(ctxs, refs))[:3]:
    break
    datasets.view_tail(datasets.actions_to_chat(ctx))
    print()
    for r in ref:
        print(r)
    print()

In [158]:
def evaluator(a):
    ctx, ref = a
    return evaluation.pointwise_gpt3(datasets.actions_to_chat(ctx), ref, return_details=True)

metrics = lmap(evaluator, list(zip(ctxs, refs)))

eta 79 s
eta 38 s
eta 19 s


In [159]:
rows = list(zip(ctxs, refs, metrics))
rows = [row for row in rows if row[2]]
rows = sorted(rows, key=lambda row: -sum([v if isinstance(v, int) else v[0] for v in row[2].values()]))

for ctx, ref, ms in rows:
    datasets.view_tail(datasets.actions_to_chat(ctx))
    print()
    for r in ref:
        print(r)

    print()
    for k, v in ms.items():
        print(str(k).ljust(10), v)
        
    print('\n' * 10)

p,chci tě jen objímat
p,snad to smyje sprcha >:)
p,~ ty >:))))
p,xd
p,dal bych si fotky 👉👈 🖤
p,👅
Robert Bates,Ňaaa 😠
Robert Bates,Dneska si nepřipadám zas tolik sexy 😢
Robert Bates,Tak 1 🥵
Robert Bates,[151946236_792021171394115_4430293381641359097_n_792021131394119.jpg]



Action(time=1613592359.579, user='p', type='reaction', data={'name': '😍', 'eos': True, 'mask_action': False})

sound      (10, '- The response fits well with the chat excerpt and the persona details provided.')
persona    (10, '- The response aligns with the persona of the 22-year-old theoretical computer science student from Czechia who is comfortable with using various emojis and slang in their conversations.')
flow       (10, '- The response contributes positively to the conversation flow by reacting to the previous messages and showing active engagement.')













Roberto Wells,how so?
p,"which is also expected, but perhaps this fact made me more prone to think critical thinking and disagreeableness are directly related"
p,"who knows, data xd"
p,but I mean
p,"If you can't stand up for your claims, you are not gonna be particularly good scientist"
p,already in IYTP / TMF
p,there is opponent role
p,sort of peer-review
p,"you give sincere critical feedback, sometimes you try to disprove someone, or you must to back yourself up"
Roberto Wells,So we could say that disagreeableness and the perception that someone is a competent critical thinker is corelated



Action(time=1611347752.593, user='p', type='message', data={'body': "if you publish a falsehood, it's in everyone's best interest for that falsehood to be disproven", 'issues': [], 'mask_action': False})
Action(time=1611347771.069, user='p', type='message', data={'body': 'particularly for the scientist who disproves it', 'issues': [], 'followup': True, 'mask_action': False})
Action(time=1611347783.15, user='p', type='message', data={'body': 'since it gains him easy reputation', 'issues': [], 'followup': True, 'eos': True, 'mask_action': False})

sound      (9, '- The response is sound and related to the discussion about publishing falsehoods in the scientific community.')
persona    (10, '- The response matches the persona given, as the theoretical computer science student is engaging in a discussion about critical thinking and disagreeableness among scientists.')
flow       (8, '- The response maintains the conversation flow by adding a new perspective on the topic being discussed. I

Daniel Mcgrath,"Which is, coincidentally, why I have so little mastery in things. lol"
p,yes
p,althouughh
p,"not necessarily, as demonstrated by jacob collier"
p,https://www.youtube.com/watch?v=t9jwpZENQUI 👍
p,the literal mozart of 21st century
p,"you can make it fun, of course"
Daniel Mcgrath,Well
Daniel Mcgrath,That's the other way
Daniel Mcgrath,"""Fuck the book I am gonna experiment 16 hours a day"""


...


Action(time=1689817478.819, user='p', type='idle', data={'eos': True, 'mask_action': False})

sound      (1, '- No, using the <IDLE> tag is not appropriate in this context and penalizing is warranted.')
persona    (1, '- No, the response does not correspond to the target persona as it lacks the expected impoliteness, wackiness, or humor.')
flow       (1, '- No, the response does not maintain the conversation flow and does not show active engagement.')













Kenneth Kim,"Fuck, man, I can't seem to be able to skim Carl Jung's Black Books smh"
Kenneth Kim,[240783949_547473223251863_4944060606541815981_n_547473219918530.png]
Joseph Ortega,About salt and alchemy http://www.aaroncheak.com/hermetic-problem-of-salt ❤
Kenneth Kim,https://www.youtube.com/watch?v=--9A2tKkk6s
Kenneth Kim,[240943995_887809112149525_3465167097193512370_n_887809108816192.png]
Kenneth Kim,[241024610_533471351270191_8946450880953622106_n_533471347936858.png]
Kenneth Kim,[241005028_890682175213771_5561050360457859390_n_890682171880438.png]
Kenneth Kim,[241020263_366944408262958_8092609910063805154_n_366944404929625.png]
Kenneth Kim,[241066483_1007721519785341_4345902765399691120_n_1007721516452008.png]
Kenneth Kim,[241114462_1244363266036543_8428832134395169036_n_1244363262703210.png]



Action(time=1630390797.31, user='p', type='idle', data={'eos': True, 'mask_action': False})

sound      (1, '(no) - The response does not correspond to the target persona as it is idling without any proper reason or reaction to the messages.')
persona    (1, '(no) - The response does not match the persona of a 22-year-old theoretical computer science student from Czechia who is usually talkative, wacky, impolite, and likes to engage in various topics.')
flow       (1, '(no) - The response does not help maintain the conversation flow as it disrupts the natural back-and-forth interaction expected in a chat scenario. It shows a lack of active engagement.')













p,and career
Victor Mcdaniel,well it's what makes people be the puer aeternus
Victor Mcdaniel,there's bitterness in it :p
Victor Mcdaniel,ye I think it's partly pursuing extrinsic values rather than intrinsic ones
Victor Mcdaniel,Kafka pog
Victor Mcdaniel,I think the Metamorphosis concerns with this productivity-centric culture pathology
Victor Mcdaniel,concerns this*
Victor Mcdaniel,i.e. the dark side of it
Victor Mcdaniel,as Gregory desperately works to provide for his loved ones (not even himself)
Victor Mcdaniel,and he's denounced by exactly those he supported as soon as he loses that productivity



Action(time=1626821072.518, user='p', type='idle', data={'eos': True, 'mask_action': False})

sound      (0, "- The response of '<IDLE>' is not appropriate given the context and the instruction to avoid idleness as much as possible.")
persona    (2, '- While the response does not show the typical characteristics of the target persona, the lack of response is not in line with being impolite, jerky, or humorous.')
flow       (0, "- The response of '<IDLE>' does not help maintain the conversation flow and shows disengagement from the conversation, contrary to the instructions provided.")











